# This notebook is used for the development of the Program Builder for the DecaDAC

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qupulse
from qupulse.pulses.plotting import plot
from qupulse.pulses import PointPT, TablePT, FunctionPT, SequencePT, RepetitionPT, AtomicMultiChannelPT, ParallelConstantChannelPT
from qupulse.serialization import Serializer, FilesystemBackend
from qupulse.hardware.setup import HardwareSetup, MeasurementMask
from qupulse.hardware.awgs.zihdawg import HDAWGTriggerOutSource

\\Janeway\User AG Bluhm\Surrey\git\qupulse\qupulse\utils\types.py:36: UserWarning: gmpy2 not found. Using fractions.Fraction as fallback. Install gmpy2 for better performance.time_from_float might produce slightly different results
  warnings.warn('gmpy2 not found. Using fractions.Fraction as fallback. Install gmpy2 for better performance.'


In [3]:
from qupulse_mfli.mfli import MFLIDAQ, postprocessing_average_within_windows

In [4]:
from qcodes.instrument_drivers.Harvard.Decadac import Decadac

In [5]:
Setup = HardwareSetup()

In [6]:
# connecting the DecaDAC
dac = Decadac('dac1', 'ASRL4::INSTR', min_val=-10, max_val=10, division=1, terminator='\n')

Connected to Harvard DecaDAC (hw ver: 0, serial: 0) in 0.49s


In [7]:
for c in dac.channels:
    c.volt(0)

In [8]:
# connecting the MFLI
mfli = MFLIDAQ.connect_to("dev7113")

prefix = ""
channels = {
    f"{prefix}R": ["demods/0/sample.R"],
    f"{prefix}T": ["demods/0/sample.Theta"],
    f"{prefix}X": ["demods/0/sample.X"],
    f"{prefix}Y": ["demods/0/sample.Y"],
    f"{prefix}A1": ["demods/0/sample.AuxIn0"],
    f"{prefix}A2": ["demods/0/sample.AuxIn1"],
    f"{prefix}F": ["demods/0/sample.Frequency"],
} 
# TODO it is possible, that the channels like to be called with *.avg at the end. At least that is what the LabOne web interface logs. But for R it worked without.

# now the channels are to be registered.
for k, v in channels.items():
    # at the lock-in side: (None refers to the default)
    mfli.register_measurement_channel(program_name=None, window_name=k, channel_path=v)
    # and at the qupulse side:
    Setup.set_measurement(f"{k}", MeasurementMask(mfli, k))

# then we set the trigger default settings
mfli.register_trigger_settings(program_name=None,
                                trigger_input=f"demods/0/sample.TrigIn1", # here TrigInN referrers to the printer label N
                                edge="rising",
                                trigger_count=1, # one trigger event per measurement
                                level=.5,
                                measurement_count=1, # one measurement per arm
                                other_settings={"holdoff/time": 1e-3}
                                )

In [9]:
import time

In [636]:
def send_trigger():
    mfli.api_session.set(f"/{mfli.serial}/auxouts/0/offset", 2)
    time.sleep(0.5)
    mfli.api_session.set(f"/{mfli.serial}/auxouts/0/offset", 0)

In [11]:
from qupulse.hardware.awgs.decadac import *

In [651]:
download_script(script_length=500, serial_ask=dac.ask_raw)

 55%|█████▌    | 138/250 [00:06<00:05, 20.83it/s]

Could not decode 10000101
Could not decode 11000011
Could not decode 10010100
Could not decode 11001110


 58%|█████▊    | 144/250 [00:06<00:05, 20.83it/s]

Could not decode 10100011
Could not decode 10000011
Could not decode 10001000


 60%|██████    | 150/250 [00:07<00:04, 20.83it/s]

...

 98%|█████████▊| 246/250 [00:11<00:00, 20.98it/s]

Could not decode 11101011
Could not decode 10001111
Could not decode 11111011
Could not decode 11010100
Could not decode 11101110
Could not decode 11010011
Could not decode 11010111


100%|██████████| 250/250 [00:11<00:00, 20.92it/s]

Could not decode 11011010
Could not decode 11111000
Could not decode 11100001
Could not decode 11001111
Could not decode 11110101
Could not decode 11001011


'*1:B0;M2;B1;M2;A1545;P32439;A1538;P3;A1577;P33095;A1570;P3;A1609;P33259;A1602;P3;A45056;P200;*2:X2050;*3:A1545;+3;A1538;P3;A1577;+-3;A1570;P3;A1609;+7;A1602;P3;$10000;*4:X772;A45056;+-1;X1283;*5:A1545;P32767;A1538;P3;A1577;P32767;A1570;P3;A1609;P32767;A1602;P3;X0;;X0;0;"(\x072$RQd p(R!\x10e\x02O\x05gP\x01 \x12",B\x04(C0,wk=\x1aW -gW8`\x0e[{<I-G[2\rS\x1bm@\tpLS(j\x11y\x1cO\x1cBL$\x1ap\x04\tD[\x05t\x05 &e=4n$\x12\\&\x05\x08\x1et|9p\x02!v\t\x0fm\x19;;5\x01ZsM{_yO!\x7f7mk?1-<'

In [633]:
# A{0x006FC1};   (Where the trigger inputs live (after hacking stuff))
# A{1545+16*(c+1)};+{int(cv)};A{1538+16*(c+1)};P3;
# (B1;C0;D{volt_to_int(0.2)};      (set the gate that we want to change to 0)

def prepare_board(board):
    return f"B{board};M2;"

def set_voltage_code(channel, volt):
    return f"A{1545+16*(channel)};P{volt_to_int(volt)};A{1538+16*(channel)};P3;"

def inc_voltage_code(channel, delta):
    int_delta = int(volt_to_int(delta)-volt_to_int(0))
    return f"A{1545+16*(channel)};+{int_delta};A{1538+16*(channel)};P3;"

def set_timer(duration_in_s):
    return f"${round(duration_in_s*1e6)};"

def not_triggered_jump(target):
    """
    SpecialConditions:
    ;08 = Trig1 is zero
    ;09 = Trig1 is not zero
    ;0A = Trig2 is zero
    ;0B = Trig2 is not zero
    """
    return f"X{0x800+target};"

def timer_running_jump(target):
    return f"X{0x300+target};"


def value_larger_zero_jump(target):
    return f"X{0x500+target};"

def unconditional_jump(target):
    return f"X{0x000+target};"

"""

  B0;C0;D{volt_to_int(0.05)};      (set the gate that we want to change to 0)
  B0;C2;D{volt_to_int(0.1)};      (set the gate that we want to change to 0)
  B1;C0;D{volt_to_int(-0.1)};      (set the gate that we want to change to 0)

  
  B0;C0;M2;G0;T0;C1;M2;G0;T0;
  B1;C0;M2;G0;T0;
  
"""
ramp_script = f"""
{{
*1:
  
  B0;M2; (setting the gating)
  B1;M2; (for all used boards)

  A45056;P200;   (Initialize our counter to 2**16-1)
  
  {set_voltage_code(0, -0.1)}
  {set_voltage_code(2, 0.1)}
  {set_voltage_code(4, 0.15)}
*2:
  X{0x802};      (X0x802 Jump to *2 if value trigger is 0)
*3:              (main loop with label "2")
  B0;C0;
  A{1545+16*(2)};+-1;     (Add one to the course DAC value for channel 0)
  A{1538+16*(2)};P3;      (Overwrite to force an update (0: no updates, 1: update enabled, 2: force update, 3: force update and update enabled))
  B0;C1;
  A{1545+16*(0)};+1;      (Add one to the course DAC value for channel 1)
  A{1538+16*(0)};P3;      (Overwrite to force an update (0: no updates, 1: update enabled, 2: force update, 3: force update and update enabled))
  B1;C0;
  A{1545+16*(4)};+2;      (Add one to the course DAC value for channel 0)
  A{1538+16*(4)};P3;      (Overwrite to force an update (0: no updates, 1: update enabled, 2: force update, 3: force update and update enabled))

  $5000;
*4:
  X{0x304};
  A45056;+ -1;   (Decrease the counter)
  X{0x503};      (X0x502 Jump to *2 if value behind the pointer is not 0)
*5:
  B0;C0;D{volt_to_int(0)};
  B0;C1;D{volt_to_int(0)};
  B1;C0;D{volt_to_int(0)};
  X0;
}};
"""

ramp_script_2 = f"""{{
*1:
  {prepare_board(0)}
  {prepare_board(1)}
  {set_voltage_code(0, -0.1)}
  {set_voltage_code(2, 0.1)}
  {set_voltage_code(4, 0.15)}
  
  A45056;P200;   (Initialize our counter)

*2:
  {not_triggered_jump(2)}
*3:
  {inc_voltage_code(0, 0.001)}
  {inc_voltage_code(2, -0.001)}
  {inc_voltage_code(4, 0.002)}

  {set_timer(0.01)}
*4:
  {timer_running_jump(4)}

  A45056;+ -1;   (Decrease the counter)
  {value_larger_zero_jump(3)}

*5:
  {set_voltage_code(0, 0)}
  {set_voltage_code(2, 0)}
  {set_voltage_code(4, 0)}
  {unconditional_jump(0)}

}};"""

In [634]:
upload_script(ramp_script_2, serial_ask=dac.ask_raw)

'X1!{*1:B0;M2;B1;M2;A1545;P32439;A1538;P3;A1577;P33095;A1570;P3;A1609;P33259;A1602;P3;A45056;P200;*2:X2050;*3:A1545;+3;A1538;P3;A1577;+-3;A1570;P3;A1609;+7;A1602;P3;$10000;*4:X772;A45056;+-1;X1283;*5:A1545;P32767;A1538;P3;A1577;P32767;A1570;P3;A1609;P32767;A1602;P3;X0;}!'

In [447]:
dac.write_raw("X0;")

In [649]:
while dac.ask("") != "": pass

In [647]:
dac.write_raw("X1;")

In [648]:
send_trigger()

In [594]:
for c in dac.channels[:5]:
    c.volt(0.15)

In [17]:
def get(addr):
    res = download_16bit(addr, serial_ask=dac.ask_raw)
    print(f"{res} -> {bin(res)}")
    return res

In [167]:
get(0x006FC1)

8058 -> 0b1111101111010


8058

In [650]:
raise NotImplementedError("This is not needed... X0x800 is designed for that...")
# inject_int(0x006FC4, 0b01001, serial_ask=dac.ask_raw) # 
inject_int(0x006F8A, 0b1111111111111001, serial_ask=dac.ask_raw) # setting the annoying registers to be output registers (0b1011101110000 & 0b1001) where the first binary number is the default
inject_int(0x006FC4, 0b1111111111111001, serial_ask=dac.ask_raw) # setting the annoying registers to 0
get(0x006FC0)

NotImplementedError: This is not needed... X0x800 is designed for that...

In [656]:
# playing around with LZ77

In [657]:
from qupulse.utils.performance import *

In [796]:
test_array = np.arange(100).reshape((50, 2))

In [797]:
def test_compression_and_reconstruction(array, intermed, diffs):
    compressed = compress_array_LZ77(array=array, allow_intermediates=intermed, using_diffs=diffs)
    uncompressed = uncompress_array_LZ77(compressed)
    if diffs:
        assert len(array) == len(uncompressed), f"{compressed=}, \n{uncompressed=}"
        assert np.allclose(array, np.cumsum(uncompressed, axis=0)), f"{compressed=}, \n{uncompressed=}"
    else:
        assert len(array) == len(uncompressed), f"{compressed=}, \n{uncompressed=}"
        assert np.allclose(array, uncompressed), f"{compressed=}, \n{uncompressed=}"
    return compressed, uncompressed

In [867]:
def test_compression_and_reconstruction_helper(array):
    _ = test_compression_and_reconstruction(array, intermed=True, diffs=True)
    _ = test_compression_and_reconstruction(array, intermed=True, diffs=False)
    _ = test_compression_and_reconstruction(array, intermed=False, diffs=True)
    _ = test_compression_and_reconstruction(array, intermed=False, diffs=False)

In [868]:
test_compression_and_reconstruction_helper(test_array)

In [885]:
test_compression_and_reconstruction_helper(np.array([[1, 2, 3, 4, 5]]).T)
test_compression_and_reconstruction_helper(np.array([[1]]))
test_compression_and_reconstruction_helper(np.ones((100, 2)))
test_compression_and_reconstruction_helper(np.ones((1, 2)))
test_compression_and_reconstruction_helper(np.ones((1, 10)))
for _ in range(10):
    test_compression_and_reconstruction_helper(np.random.uniform(0, 1, (np.random.randint(1, 10), np.random.randint(1, 10))))


In [900]:
special_array = np.repeat(np.linspace(0, 100, 200)[:, None], 3, axis=1)
special_array[:, 0] = special_array[:, 0]%5*0.1
special_array[:, 1] = special_array[:, 1]%10*3

In [901]:
special_array = special_array.astype(int)

In [902]:
test_compression_and_reconstruction_helper(special_array.astype(int))
comp, _ = test_compression_and_reconstruction(special_array.astype(int), intermed=False, diffs=True)
comp

[(0, 0, array([0, 0, 0])),
 (0, 0, array([0, 1, 0])),
 (0, 0, array([0, 2, 1])),
 (2, 17, array([  0, -28,   1])),
 (20, 46, array([0, 2, 0])),
 (0, 0, array([0, 1, 1])),
 (2, 11, array([  0, -29,   1])),
 (0, 0, array([0, 2, 0])),
 (0, 0, array([0, 1, 1])),
 (2, 17, array([  0, -29,   1])),
 (20, 33, array([0, 2, 1])),
 (0, 0, array([0, 1, 0])),
 (2, 4, array([  0, -28,   1])),
 (0, 0, array([0, 1, 0])),
 (0, 0, array([0, 2, 1])),
 (2, 17, array([  0, -28,   1])),
 (20, 38, array([  0, -28,   1]))]

In [906]:
csd_2d = np.meshgrid(np.linspace(0, 1, 10), np.linspace(0, 1, 7), indexing='ij')
csd_2d_with_sensor = np.ones((3, 10, 7))
csd_2d_with_sensor[0] = csd_2d[0]*0.1+0.51
csd_2d_with_sensor[1] = csd_2d[1]*0.5+0.322
csd_2d_with_sensor[2] = -1*csd_2d_with_sensor[0]*0.1+-1*csd_2d_with_sensor[1]*0.14
csd_2d_with_sensor.shape

(3, 10, 7)

In [908]:
csd_pulse = csd_2d_with_sensor.reshape((3, -1)).T*100

In [910]:
csd_pulse.astype(int)

array([[ 51,  32,  -9],
       [ 51,  40, -10],
       [ 51,  48, -11],
       [ 51,  57, -13],
       [ 51,  65, -14],
       [ 51,  73, -15],
       [ 51,  82, -16],
       [ 52,  32,  -9],
       [ 52,  40, -10],
       [ 52,  48, -12],
       [ 52,  57, -13],
       [ 52,  65, -14],
       [ 52,  73, -15],
       [ 52,  82, -16],
       [ 53,  32,  -9],
       [ 53,  40, -10],
       [ 53,  48, -12],
       [ 53,  57, -13],
       [ 53,  65, -14],
       [ 53,  73, -15],
       [ 53,  82, -16],
       [ 54,  32,  -9],
       [ 54,  40, -11],
       [ 54,  48, -12],
       [ 54,  57, -13],
       [ 54,  65, -14],
       [ 54,  73, -15],
       [ 54,  82, -16],
       [ 55,  32, -10],
       [ 55,  40, -11],
       [ 55,  48, -12],
       [ 55,  57, -13],
       [ 55,  65, -14],
       [ 55,  73, -15],
       [ 55,  82, -17],
       [ 56,  32, -10],
       [ 56,  40, -11],
       [ 56,  48, -12],
       [ 56,  57, -13],
       [ 56,  65, -14],
       [ 56,  73, -15],
       [ 56,  82

In [909]:
test_compression_and_reconstruction_helper(csd_pulse.astype(int))
comp, _ = test_compression_and_reconstruction(csd_pulse.astype(int), intermed=False, diffs=True)
comp

[(0, 0, array([51, 32, -9])),
 (0, 0, array([ 0,  8, -1])),
 (1, 1, array([ 0,  9, -2])),
 (0, 0, array([ 0,  8, -1])),
 (1, 1, array([ 0,  9, -1])),
 (0, 0, array([  1, -50,   7])),
 (0, 0, array([ 0,  8, -1])),
 (0, 0, array([ 0,  8, -2])),
 (0, 0, array([ 0,  9, -1])),
 (7, 11, array([ 0,  8, -2])),
 (0, 0, array([ 0,  8, -1])),
 (0, 0, array([ 0,  9, -1])),
 (0, 0, array([ 0,  8, -1])),
 (1, 1, array([ 0,  9, -1])),
 (0, 0, array([  1, -50,   6])),
 (0, 0, array([ 0,  8, -1])),
 (1, 1, array([ 0,  9, -1])),
 (0, 0, array([ 0,  8, -1])),
 (1, 1, array([ 0,  9, -2])),
 (0, 0, array([  1, -50,   7])),
 (7, 11, array([ 0,  8, -2])),
 (0, 0, array([ 0,  9, -1])),
 (0, 0, array([  1, -50,   7])),
 (0, 0, array([ 0,  8, -1])),
 (1, 1, array([ 0,  9, -1])),
 (0, 0, array([ 0,  8, -2])),
 (0, 0, array([ 0,  8, -1])),
 (7, 8, array([  2, -50,   7])),
 (0, 0, array([ 0,  8, -1])),
 (1, 1, array([ 0,  9, -2])),
 (0, 0, array([ 0,  8, -1])),
 (1, 1, array([ 0,  9, -1]))]

In [957]:
vg_csd_2d = np.meshgrid(np.linspace(0, 1, 4), np.linspace(0, 1, 100), indexing='ij')
vg_csd_2d_with_sensor = np.ones((2, np.prod(vg_csd_2d[0].shape)))
vg_csd_2d_with_sensor[0] = (vg_csd_2d[0].flatten()*0.1+0.51)*+0.9 + (vg_csd_2d[1].flatten()*0.5+0.322)*-0.1
vg_csd_2d_with_sensor[1] = (vg_csd_2d[0].flatten()*0.1+0.51)*-0.2 + (vg_csd_2d[1].flatten()*0.5+0.322)*+0.8
vg_csd_pulse = vg_csd_2d_with_sensor.T*100
vg_csd_pulse = np.floor(vg_csd_pulse)
print(f"{vg_csd_pulse.shape=}")

# test_compression_and_reconstruction_helper(vg_csd_pulse.astype(int))
comp, _ = test_compression_and_reconstruction(vg_csd_pulse.astype(int), intermed=False, diffs=True)
len(comp), comp

vg_csd_pulse.shape=(400, 2)
[0] [0]
(0, 0, array([0, 0]))
[0] [0]
(0, 0, array([0, 1]))
[0, 2] [0, 3]
(2, 2, array([0, 0]))
[0, 1, 3, 5] [0, 1, 3, 5]
(5, 5, array([0, 1]))
[0, 2, 4, 6, 7, 9, 11] [0, 1, 1, 1, 2, 1, 1]
(0, 0, array([0, 0]))
[0, 1, 3, 5, 7, 8, 10, 12] [0, 1, 1, 1, 1, 1, 1, 1]
(1, 1, array([-1,  1]))
[0, 2, 3, 5, 7, 9, 10, 12, 14] [0, 1, 1, 4, 3, 3, 1, 8, 3]
(0, 0, array([0, 0]))
[0, 5, 7, 9, 12, 14] [0, 3, 2, 2, 7, 2]
(0, 0, array([0, 1]))
[0, 2, 4, 5, 7, 9, 11, 12, 14, 16] [0, 1, 1, 2, 1, 1, 1, 6, 1, 1]
(0, 0, array([0, 0]))
[0, 1, 3, 5, 6, 8, 10, 12, 13, 15, 17] [0, 1, 3, 1, 1, 3, 6, 5, 1, 3, 10]
(3, 3, array([0, 1]))
[0, 2, 4, 5, 7, 9, 10, 12, 14, 16, 17, 19, 21] [0, 1, 1, 12, 1, 1, 2, 1, 1, 1, 6, 1, 1]
(5, 10, array([0, 0]))
[0, 1, 3, 5, 6, 8, 10, 11, 13, 15, 16, 18, 20, 21, 23, 25, 27, 28, 30, 32] [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 30, 1, 1, 1, 1, 1, 1, 1]
(20, 20, array([0, 0]))
[0, 20, 40] [0, 9, 9]
(0, 0, array([-1,  1]))
[0, 2, 3, 5, 7, 8, 10, 12, 13, 15, 17, 1

(107,
 [(0, 0, array([42, 15])),
  (0, 0, array([0, 0])),
  (0, 0, array([0, 1])),
  (2, 2, array([0, 0])),
  (5, 5, array([0, 1])),
  (0, 0, array([0, 0])),
  (1, 1, array([-1,  1])),
  (0, 0, array([0, 0])),
  (0, 0, array([0, 1])),
  (0, 0, array([0, 0])),
  (3, 3, array([0, 1])),
  (5, 10, array([0, 0])),
  (20, 20, array([0, 0])),
  (0, 0, array([-1,  1])),
  (0, 0, array([0, 0])),
  (0, 0, array([0, 1])),
  (0, 0, array([0, 0])),
  (3, 3, array([0, 1])),
  (7, 7, array([0, 0])),
  (3, 3, array([-1,  1])),
  (0, 0, array([0, 0])),
  (1, 1, array([0, 1])),
  (2, 2, array([0, 0])),
  (20, 20, array([  8, -41])),
  (0, 0, array([0, 1])),
  (0, 0, array([0, 0])),
  (2, 2, array([0, 0])),
  (5, 5, array([0, 1])),
  (2, 2, array([-1,  0])),
  (0, 0, array([0, 0])),
  (0, 0, array([0, 1])),
  (2, 2, array([0, 0])),
  (5, 10, array([0, 1])),
  (0, 0, array([0, 0])),
  (1, 1, array([0, 1])),
  (0, 0, array([-1,  0])),
  (0, 0, array([0, 1])),
  (0, 0, array([0, 0])),
  (1, 1, array([0, 1])